In [1]:
# DATA PREPARATION

import pandas as pd
import numpy as np

# Load data from CSV file
try:
    df = pd.read_csv('../data/synthetic/synthetic_network.csv')
except FileNotFoundError:
    print("Error: File not found. Please check the file path.")
    exit()
except pd.errors.EmptyDataError:
    print("Error: The file is empty or invalid.")
    exit()


# Identify unique nodes and create a mapping for indices
nodos = sorted(set(df['origen']).union(set(df['destino'])))
node_to_index = {node: idx for idx, node in enumerate(nodos)}
n = len(nodos)

# Initialize the cost matrix with infinity
cost_matrix = np.full((n, n), np.inf)


# Set diagonal to 0 (self-costs)
np.fill_diagonal(cost_matrix, 0)

# Fill the cost matrix with the values from the CSV
for _, row in df.iterrows():
    try:
        origen = row['origen']
        destino = row['destino']
        costo = float(row['costo'])
        cost_matrix[node_to_index[origen], node_to_index[destino]] = costo
    except KeyError:
        print("Error: Missing columns 'origen', 'destino', or 'costo'.")
        exit()
    except ValueError:
        print(f"Error: Invalid cost value on row {_}.")
        exit()

# Display the cost matrix
print("Cost Matrix:")
print(cost_matrix)

np.save('../data/synthetic/cost_matrix.npy', cost_matrix)




Cost Matrix:
[[  0.  93.  inf 184.  15.  inf  86. 166.  inf  inf]
 [164.   0.  inf  inf 169.  inf  inf  inf 126. 177.]
 [151.  inf   0.  inf 163.  inf  16.  inf 114. 184.]
 [ inf  inf  inf   0.  16.  inf 103.  inf 149.  inf]
 [ inf  inf  inf  inf   0.  inf  inf  61.  62.  inf]
 [ 84.  inf  97.  inf 151.   0.  inf  inf 177.  inf]
 [ 78.  91.  inf  60.  29. 195.   0. 132.  20.  54.]
 [128.  inf  31.  inf  57.  28. 187.   0. 133.  49.]
 [ inf  inf 196. 117. 111.  22. 132.  36.   0.  inf]
 [ inf  inf  inf  inf  20.  inf  inf 145.  inf   0.]]


Offline training to learn graph properties without source/destination nodes

In [5]:
import tensorflow as tf
import numpy as np

# Placeholder for cost matrix
C = cost_matrix
n = C.shape[0]  # Number of nodes
C_flat = C.flatten()
C_flat[np.isinf(C_flat)] = 1e6  # Replace infinity with a large value

# Offline training: Learn graph properties without source/destination nodes
def offline_loss(y_true, y_pred):
    arc_values = y_pred[:, :n * n]
    # Path cost
    term1 = tf.reduce_sum(C_flat * arc_values)
    # Outgoing edge constraint
    row_sums = tf.reduce_sum(tf.reshape(arc_values, (-1, n, n)), axis=2) - 1
    term2 = tf.reduce_sum(tf.square(row_sums))
    # Incoming edge constraint
    col_sums = tf.reduce_sum(tf.reshape(arc_values, (-1, n, n)), axis=1) - 1
    term3 = tf.reduce_sum(tf.square(col_sums))
    # Binary values constraint
    term4 = tf.reduce_sum(arc_values * (1 - arc_values))
    loss = term1 + 10 * term2 + 10 * term3 + 10 * term4
    return loss

# Define offline training model
offline_input_dim = len(C_flat)
offline_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(offline_input_dim,)),
    tf.keras.layers.Dense(offline_input_dim, activation='sigmoid')
])

# Compile offline model
offline_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                      loss=offline_loss)

# Train offline model
offline_input = np.expand_dims(C_flat, axis=0)
offline_model.fit(offline_input, offline_input, epochs=500, verbose=1)

# Save pre-trained model
offline_model.save('../models/offline_model.h5')




Epoch 1/500


/home/roberto/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - loss: 23006156.0000
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 23006156.0000
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 23006156.0000
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 23006156.0000
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 23006156.0000
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 23006156.0000
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 23006156.0000
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 23006156.0000
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 23006156.0000
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 23006156.0000
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 23006156.0000
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 23006156.0000
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 23006156.0000
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 23006156.000

Fine-tune pretrained model for specific origin and destination

In [6]:
import tensorflow as tf
import numpy as np

origin = 0
destination = 3

# Load pre-trained offline model
offline_model = tf.keras.models.load_model('../models/offline_model.h5',
                                           custom_objects={'offline_loss': offline_loss})

# Placeholder for cost matrix and input vector creation
C = cost_matrix
n = C.shape[0]
C_flat = C.flatten()
C_flat[np.isinf(C_flat)] = 1e6

# Create input vector with source and destination nodes
def create_input_vector(C_flat, origin, destination, n):
    origin_vector = np.zeros(n)
    origin_vector[origin] = 1
    destination_vector = np.zeros(n)
    destination_vector[destination] = 1
    return np.concatenate([C_flat, origin_vector, destination_vector])

# Fine-tuning for specific source and destination nodes
def energy_loss_with_input_vectors(y_true, y_pred):
    arc_values = y_pred[:, :n * n]
    origin_vector = y_pred[:, n * n:n * n + n]
    destination_vector = y_pred[:, n * n + n:]
    term1 = tf.reduce_sum(C_flat * arc_values)
    row_sums = tf.reduce_sum(tf.reshape(arc_values, (-1, n, n)), axis=2) - 1
    term2 = tf.reduce_sum(tf.square(row_sums))
    col_sums = tf.reduce_sum(tf.reshape(arc_values, (-1, n, n)), axis=1) - 1
    term3 = tf.reduce_sum(tf.square(col_sums))
    term4 = tf.reduce_sum(arc_values * (1 - arc_values))
    s = tf.argmax(origin_vector, axis=1)
    arc_matrix = tf.reshape(arc_values, (-1, n, n))
    source_out = tf.gather(arc_matrix, s, batch_dims=1)
    source_constraint = tf.reduce_sum(source_out, axis=1) - 1
    term5 = tf.reduce_sum(tf.square(source_constraint))
    d = tf.argmax(destination_vector, axis=1)
    dest_in = tf.gather(tf.transpose(arc_matrix, perm=[0, 2, 1]), d, batch_dims=1)
    dest_constraint = tf.reduce_sum(dest_in, axis=1) - 1
    term6 = tf.reduce_sum(tf.square(dest_constraint))
    loss = term1 + 10 * term2 + 10 * term3 + 10 * term4 + 10 * term5 + 10 * term6
    return loss

# Define full model for fine-tuning
input_dim = len(C_flat) + 2 * n
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_dim,)),
    tf.keras.layers.Dense(input_dim, activation='sigmoid')
])

# Load weights from offline model and adjust
offline_weights = offline_model.layers[0].get_weights()
offline_kernel, offline_bias = offline_weights

# Adjust weights to match the new input dimension
new_kernel = np.zeros((input_dim, input_dim))  # Create new kernel with the correct size
new_bias = np.zeros(input_dim)  # Adjust bias size

# Copy offline weights for the C_flat portion
new_kernel[:len(C_flat), :len(C_flat)] = offline_kernel
new_bias[:len(C_flat)] = offline_bias

# Set adjusted weights into the fine-tuning model
model.layers[0].set_weights([new_kernel, new_bias])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=energy_loss_with_input_vectors)

# Create input vector
input_vector = create_input_vector(C_flat, origin, destination, n)
input_data = np.expand_dims(input_vector, axis=0)

# Fine-tune the model
model.fit(input_data, input_data, epochs=500, verbose=1)

# Predictions and results
predictions = model.predict(input_data)[0]
arc_values = predictions[:n * n]
arc_matrix = arc_values.reshape((n, n)) > 0.5

print("Matriz de arcos seleccionados:")
print(arc_matrix.astype(int))

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step - loss: 23006286.0000
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 23006286.0000
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 23006286.0000
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 23006286.0000
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 23006286.0000
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 23006286.0000
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 23006286.0000
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 23006286.0000
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 23006286.0000
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 23006286.0000
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 23006286.0000
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 23006286.0000
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 23006286.0000
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 